### Transformers, General Python, and Gradio Exercices

---


### First part

1) Install the library `gradio`, and the library `transformers`.

2) Create a `gradio` interface connected to GPT-2, like we did during the course. Use a `max_length` of `100`, `10` `num_beams`, `early_stopping=True`, and `no_repeat_ngram_size=2`.

3) Ask the question "Who are you?"

4) Why is the model not answering "I am GPT-2, a ChatBot designed to answer your questions". Well because, it is not a ChatBot designed to answer your questions yet, for now it is (roughly speaking) just guessing what is most likely next word based on the previous ones.

5) Modify your `gradio` function to include, for every text inputted by the user, a sentence or two to steer the model into behaving like a ChatBot.

6) Now, you would probably want to hide the sentence(s) you have added to steer the model to behave like you wanted to. Modify `generated_text` so that it only displays the conversation and not the 'wrapper' indicating to the model how to behave.

---

### Second Part

7) Let's now try to use a more recent model: `Llama`. Run the following code:

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git@refs/pull/25740/head accelerate

`git+https://github.com/huggingface/transformers.git@refs/pull/25740/head`: This is a direct link to a specific pull request (`PR #25740`) from the `transformers` repository on GitHub. This means you're not installing the main version of the `transformers` library that's available on PyPI (Python Package Index) but a specific version from this PR.

Using `git+` followed by a GitHub URL is a way to tell `pip` to install a Python package directly from a Git repository.

This is not too important if you don't fully understand it.

You may need to restart your runtime since we had already installed another version of the `Transformers` library. Do not execute the previous cells otherwise it will ask you to re-restart your runtime. Only execute the cells from this second part of the exercise. Start a GPU Runtime otherwise it will not work. To change it, click on Change runtime type and you will arrive here: https://i.imgur.com/8QjVO0P.png. Select T4 GPU. Why is this useful? This is because on colab, the GPU has more RAM, and you need *a lot* of RAM to load big models.

Executing the following cell might take some time. Try to understand every line there is in the meantime.

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch

model = "codellama/CodeLlama-7b-Instruct-hf" # "codellama/CodeLlama-7b-hf"
# Code Llama is a collection of pretrained and fine-tuned generative text models
# ranging in scale from 7 billion to 34 billion parameters.
# We are here using the 7 billion parameters version.

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

prompt = "Who are you?"

sequences = pipeline(
    prompt,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=100,
    no_repeat_ngram_size=2,  # avoids repeating the same ngrams
    add_special_tokens=False
)

for seq in sequences:
    print(f"Result: {seq['generated_text']}")

8) Why are we using `torch.float16`?

Using float16 (also known as "half precision") can lead to faster processing times but might compromise a bit on precision.

9) Why is there a for loop:
`for seq in sequences:`?

What can you deduce about the `sequences` object returned? Prove it using the `type` built-in Python function. **Careful, when you want to re-use Llama, do not re-load the whole pipeline! You don't need to load the pipeline everytime, once it's loaded, you just need to call it.**

What if we want the `list` sequences to have more than one element? Print the `k` sequences you asked the model to generate. Comment. If you do not specify `num_beams` > 1, it will not work (for obvious reasons). The diference might be subtle (only one or a few words).

We have the same problem as we had with GPT-2. We did to encapsulate the prompt with some "instructions".

This is a lot better!

Now let's incorporate Llama in `gradio`, and try to benchmark `GPT-2` and `Llama-7b` by creating 10 basic questions on a subject you are good at (it could be translating a sentence from Spanish to English, or a question about physics, ...) and for each question, ask both ChatBots, and say if they were good or not in a dataframe with four columns: Question, Answer, Correct GPT-2 (boolean), Correct Llama.

Then save your dataframe as a `.csv` and concatenate all the `.csv` with your teammates. And determine a %accuracy on your benchmark for both models.

Obviously the quality of the responses might depend on other factors like `num_beams`, etc ... you can test different `num_beams` as well. But for a unified benchmark across the class, it's better to agree on a set number for each ChatBot.